In [1]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib import animation, rc
from IPython.display import HTML

In [2]:
gen_images = np.load("../drawing/simulated/generated/samples.npy")
gen_latent = np.load("../drawing/simulated/generated/latent.npy")


In [3]:
print("images_shape ", gen_images.shape)
print("latent_shape", gen_latent.shape)

images_shape  (10, 50, 16384)
latent_shape (10, 50, 10)


In [4]:
def sigmoid(x):
    return 1/(1 +np.exp(-x))


In [5]:

def init():
    # make an empty frame
    
    image.set_data([[0]*128, [0]*128])    
    axs[1].boxplot([])
    
    return image, 

In [6]:
def animate(i):
    img = sigmoid(gen_images[-1, i, :].reshape((128, 128)))
    latent = gen_latent[:, i, :]
    
    axs[1].cla()
    axs[1].set_ylim((-2.5, 4))
    axs[1].boxplot(x = latent)
    image.set_data(img)
    
    return image,

In [7]:
%%capture 

fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(15, 10))
image = axs[0].imshow(sigmoid(gen_images[-1, [0]].reshape((128, 128))), cmap="Greys_r")
fig.suptitle("Animation of generated events and their associated latent expressions")


axs[1].boxplot([])
axs[1].set_ylim((-2.5, 4))

anim = animation.FuncAnimation(
    fig,
    animate,
    np.arange(gen_images.shape[1]),
    init_func=init,
    interval=4000
    )


In [8]:
HTML(anim.to_html5_video())